#  Spaceship Titanic
#### In this competition your task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system.

In [1]:
import polars as pl
import numpy as np

In [2]:
#importing dataset
train_data=pl.read_csv("dataset_vs/train.csv")
test_data=pl.read_csv("dataset_vs/test.csv")
train_data.describe()

describe,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
str,str,str,f64,str,str,f64,f64,f64,f64,f64,f64,f64,str,f64
"""count""","""8693""","""8693""",8693.0,"""8693""","""8693""",8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,"""8693""",8693.0
"""null_count""","""0""","""201""",217.0,"""199""","""182""",179.0,203.0,181.0,183.0,208.0,183.0,188.0,"""200""",0.0
"""mean""",null,null,0.358306,null,null,28.82793,0.023439,224.687617,458.077203,173.729169,311.138778,304.854791,null,0.503624
"""std""",null,null,0.47953,null,null,14.489021,0.151303,666.717663,1611.48924,604.696458,1136.705535,1145.717189,null,0.500016
"""min""","""0001_01""","""Earth""",0.0,"""A/0/P""","""55 Cancri e""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""Aard Curle""",0.0
"""max""","""9280_02""","""Mars""",1.0,"""T/3/P""","""TRAPPIST-1e""",79.0,1.0,14327.0,29813.0,23492.0,22408.0,24133.0,"""Zubeneb Pashar…",1.0
"""median""",null,null,0.0,null,null,27.0,0.0,0.0,0.0,0.0,0.0,0.0,null,1.0
"""25%""",null,null,null,null,null,19.0,null,0.0,0.0,0.0,0.0,0.0,null,null
"""75%""",null,null,null,null,null,38.0,null,47.0,76.0,27.0,59.0,46.0,null,null


In [3]:
train_data.head(5)

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
str,str,bool,str,str,f64,bool,f64,f64,f64,f64,f64,str,bool
"""0001_01""","""Europa""",false,"""B/0/P""","""TRAPPIST-1e""",39.0,false,0.0,0.0,0.0,0.0,0.0,"""Maham Ofraccul…",false
"""0002_01""","""Earth""",false,"""F/0/S""","""TRAPPIST-1e""",24.0,false,109.0,9.0,25.0,549.0,44.0,"""Juanna Vines""",true
"""0003_01""","""Europa""",false,"""A/0/S""","""TRAPPIST-1e""",58.0,true,43.0,3576.0,0.0,6715.0,49.0,"""Altark Susent""",false
"""0003_02""","""Europa""",false,"""A/0/S""","""TRAPPIST-1e""",33.0,false,0.0,1283.0,371.0,3329.0,193.0,"""Solam Susent""",false
"""0004_01""","""Earth""",false,"""F/1/S""","""TRAPPIST-1e""",16.0,false,303.0,70.0,151.0,565.0,2.0,"""Willy Santanti…",true


In [4]:
train_data.null_count()

PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,201,217,199,182,179,203,181,183,208,183,188,200,0


In [5]:
# Defning target values and Organizing training data
X=train_data.clone().sample(len(train_data),shuffle=True)
X_test=test_data.clone()
y=X["Transported"]
X=X.drop(["PassengerId","Name","Transported"])
X_test=X_test.drop(["PassengerId","Name"])


In [6]:
def cabin(cab_col):
    cabin=[]
    for i in cab_col:
        if(i!=None):
            cabin.append(i[0]+i[-1])
        else:
            cabin.append(None)
    return cabin

In [7]:
# Managing Categorical and Boolean values

#Boolean - 
X=X.with_columns(X["CryoSleep"].cast(int))
X=X.with_columns(X["VIP"].cast(int))

X_test=X_test.with_columns(X_test["CryoSleep"].cast(int))
X_test=X_test.with_columns(X_test["VIP"].cast(int))

y=y.cast(int)

#String - 
#Cabin_df=pl.DataFrame({"Cabin":cabin(X["Cabin"])})
#Cabin_df_test=pl.DataFrame({"Cabin":cabin(X_test["Cabin"])})

X=X.drop("Cabin")
X_test=X_test.drop("Cabin")

#X=pl.concat([X,Cabin_df],how="horizontal")
#X=pl.concat([X_test,Cabin_df_test],how="horizontal")


import polars.selectors as cs
#train_data.select([pl.col(pl.NUMERIC_DTYPES)])
cat_col=X.select(cs.string()).columns
X[cat_col].head()

from sklearn.preprocessing import OneHotEncoder
oh_encoder=OneHotEncoder(handle_unknown="ignore",sparse=False)

oh_X = pl.DataFrame(oh_encoder.fit_transform(X[cat_col]))
X_num=pl.concat([X,oh_X],how="horizontal").drop(cat_col)

oh_X_test = pl.DataFrame(oh_encoder.fit_transform(X_test[cat_col]))
X_num_test=pl.concat([X_test,oh_X_test],how="horizontal").drop(cat_col)



/home/fansan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/home/fansan/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
print((X_num["CryoSleep"]==0).sum())
print((X_num["VIP"]==0).sum())

5439
8291


In [9]:
# Handling Null values
from sklearn.impute import SimpleImputer

X_num=X_num.with_columns(X_num["Age"].fill_nan(X_num["Age"].median()))
X_num=X_num.with_columns(X_num["CryoSleep"].fill_nan(1))
X_num=X_num.with_columns(X_num["VIP"].fill_nan(0))

imputer=SimpleImputer(strategy="median")
X_imputed=pl.DataFrame(imputer.fit_transform(X_num))
X_imputed.columns=X_num.columns

X_imputed_test=pl.DataFrame(imputer.fit_transform(X_num_test))
X_imputed_test.columns=X_num_test.columns



In [10]:
X_imputed.describe()

describe,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0,8693.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",0.349362,28.790291,0.022892,220.009318,448.434027,169.5723,304.588865,298.26182,0.529391,0.24514,0.202347,0.023122,0.207063,0.091568,0.680433,0.020936
"""std""",0.476796,14.341404,0.149568,660.51905,1595.790627,598.007164,1125.562559,1134.126417,0.499164,0.430195,0.401772,0.1503,0.405224,0.288432,0.466336,0.14318
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""max""",1.0,79.0,1.0,14327.0,29813.0,23492.0,22408.0,24133.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""median""",0.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
"""25%""",0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""75%""",1.0,37.0,0.0,41.0,61.0,22.0,53.0,40.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
X_imputed_test.describe()

describe,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,column_0,column_1,column_2,column_3,column_4,column_5,column_6,column_7
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0,4277.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",0.361001,28.60159,0.017302,215.062427,428.592238,173.233107,295.895955,304.898293,0.529109,0.234276,0.216273,0.020341,0.196633,0.090718,0.691139,0.02151
"""std""",0.480347,14.032629,0.130409,601.914503,1510.155974,554.991776,1104.872018,1235.991811,0.49921,0.423595,0.411751,0.141182,0.397499,0.287241,0.462078,0.145095
"""min""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""max""",1.0,79.0,1.0,11567.0,25273.0,8292.0,19844.0,22272.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
"""median""",0.0,26.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
"""25%""",0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""75%""",1.0,37.0,0.0,48.0,66.0,27.0,43.0,31.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Spliting data
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X_imputed,y,train_size=0.9,test_size=0.1, random_state=42)
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

(7823, 16)
(7823,)
(870, 16)
(870,)


In [13]:
from sklearn.metrics import mean_absolute_error
from xgboost import XGBClassifier

dic={}
for i in np.arange(0.01,1,0.01):
    model_xg=XGBClassifier(n_estimators=1500,eta=i,random_state=42,max_depth=8)
    model_xg.fit(X_train,y_train,
                early_stopping_rounds=20,
                eval_set=[(X_valid,y_valid)],
                verbose=False)
    score=model_xg.score(X_valid,y_valid)
    print(i,score)
    dic[i]=score

/home/fansan/.local/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.01 0.7931034482758621


/home/fansan/.local/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.02 0.7896551724137931


/home/fansan/.local/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

0.97 0.7816091954022989


/home/fansan/.local/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.98 0.7873563218390804


/home/fansan/.local/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


0.99 0.7816091954022989


In [14]:
model_xg=XGBClassifier(n_estimators=1500,eta=i,random_state=42,max_depth=8)
model_xg.fit(X_train,y_train,
                early_stopping_rounds=20,
                eval_set=[(X_valid,y_valid)],
                verbose=False)

/home/fansan/.local/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.99, eval_metric=None,
              feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=1500, n_jobs=None,
              num_parallel_tree=None, predictor=None, ...)

In [15]:
model_xg.score(X_valid,y_valid)

0.7816091954022989

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier


# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid, n_estimator):
    model1 = RandomForestClassifier(n_estimators=n_estimator, random_state=42,max_depth=8,max_samples=0.3)
    model2=GradientBoostingClassifier(n_estimators=n_estimator,random_state=42,max_depth=8)
    model2.fit(X_train, y_train)
    preds = model2.predict(X_valid)
    return model2,mean_absolute_error(y_valid, preds)


In [ ]:
mae_dic={}
for i in range(100,1000,100):
    model,mae=score_dataset(X_train,X_valid,y_train,y_valid,i)
    acc=model.score(X_valid,y_valid)
    mae_dic[i]=mae
    print(mae,'\t',acc,'\t',i)


0.2103448275862069 	 0.7896551724137931 	 100
0.20919540229885059 	 0.7908045977011494 	 200
0.21264367816091953 	 0.7873563218390804 	 300
0.21264367816091953 	 0.7873563218390804 	 400
0.21724137931034482 	 0.7827586206896552 	 500
0.2149425287356322 	 0.7850574712643679 	 600
0.21839080459770116 	 0.7816091954022989 	 700


In [ ]:
best_n=min(mae_dic,key=mae_dic.get)
best_nAdaBoostClassifier

In [ ]:
# The Final Model will be 
model=RandomForestClassifier(n_estimators=best_n,random_state=42,max_depth=8,max_samples=0.3)
model.fit(X_imputed,y)
model

In [60]:
prediction=model_xg.predict(X_valid)
test_prediction=model_xg.predict(X_imputed_test)
test_prediction=test_prediction.astype(bool)


In [61]:
test_prediction

array([ True, False,  True, ...,  True,  True, False])

In [62]:
import pandas as pd
output = pd.DataFrame({'PassengerId': test_data["PassengerId"].to_numpy().astype(str),
                       'Transported': test_prediction})
output.to_csv('submission.csv',index=False)


In [63]:
#import pathlib
#path: pathlib.Path ="dataset_vs/submission.csv"
#output.write_csv("submission.csv", separator=",")

In [64]:
output

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [65]:
model_xg.score(X_imputed,y)

0.8443575290463592